In [1]:
import torch
from models import MLP
import os

In [2]:
deepseed_model = MLP(stages=1)
adag_model = MLP(stages=1)

In [3]:
step = 100
deepseed_model.load_state_dicts(f"/mnt/local_storage/mlp_benchmark/deepspeed/global_step{step}")

# net_new.load_state_dicts("ckpt_new/global_step1")

Model state dictionaries loaded successfully!


In [4]:
def update_state_dict_keys(state_dict):
    updated_state_dict = {}
    for key, value in state_dict.items():
        new_key = f"model.{key}"
        updated_state_dict[new_key] = value
    return updated_state_dict

In [5]:
adag_state_dict = torch.load(f"/mnt/local_storage/mlp_benchmark/adag/step{step}_model.pth")
adag_state_dict = update_state_dict_keys(adag_state_dict)

In [6]:
adag_model.load_state_dict(adag_state_dict)

<All keys matched successfully>

In [8]:
mismatch = False
# Compare the values in each layer
for (adag_name, adag_param), (ds_name, ds_param) in zip(adag_model.named_parameters(), deepseed_model.named_parameters()):
    if torch.allclose(adag_param, ds_param, atol=1e-4):
        print(f"Layer {ds_name}: Match")
    else:
        difference = adag_param - ds_param
        print(f"Layer {ds_name}: Mismatch")
        print(f"Max Difference: {difference.max():.6f}")

Layer model.0.weight: Match
Layer model.0.bias: Match
Layer model.2.weight: Match
Layer model.2.bias: Match
Layer model.4.weight: Match
Layer model.4.bias: Match
Layer model.6.weight: Match
Layer model.6.bias: Match
Layer model.8.weight: Match
Layer model.8.bias: Match
Layer model.10.weight: Match
Layer model.10.bias: Match
Layer model.12.weight: Match
Layer model.12.bias: Match
Layer model.14.weight: Match
Layer model.14.bias: Match
Layer model.16.weight: Match
Layer model.16.bias: Mismatch
Max Difference: 0.000538
